# Humans or Horses...

Here we build a Convolution Neural Network to distinguish whether the content of an image is human or a horse. The dataset used for this project contains over a thousand 300x300 pixel images of horses and humans in different poses. This notebook uses the steps in the book **AI and Machine Learning for Coders**, chapter 3 from page 42, as a guide

### First fetch the dataset and visualize a few images